# Finetuning DistilBERT Model for Toxic Comment Classification

### Importing Libraries
*NOTE: Transformers version>4.20 necessary for use of Trainer, TrainingArguments, and DistilBertTokenizerFast*

In [1]:
%pip install transformers==4.2
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader 
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.2.0-py3-none-any.whl (1.8 MB)
  Using cached tokenizers-0.9.4.tar.gz (184 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      Traceback (most recent call last):
        File "/usr/lib/python3/dist-packages/pip/_vendor/pep517/in_process/_in_process.py", line 363, in <module>
          main()
        File "/usr/lib/python3/dist-packages/pip/_vendor/pep517/in_process/_in_process.py", line 345, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
        File "/usr/lib/python3/dist-packages/pip/_vendor/pep517/in_process/_in_process.py", line 130, in get_requires_for_build_wheel
          return hook(config_settings)
        File "/usr/lib/python3/dist-packages/setuptools

/home/yohann/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setting Up For GPU Usage

In [2]:
from torch import cuda
device = torch.device('cuda' if cuda.is_available() else 'cpu')

print(f"Current device: {device}")

Current device: cuda


## Preprocessing and Cleaning Domain Data
*Preprocessing assumes that both csv files are downloaded, unzipped, and saved in data/input/~.csv*

1. Read the csv files into dataframes using Pandas.
2. Drop the ID column from the data.
3. Take all the values of individual categories of toxicity and combine into a new column: 'labels'.
4. Drop all the old columns of individual categories.
5. Change all comment text to lower case.
6. Replace non-breaking spaces with regular spaces and ensure there is only one space between each word.

In [3]:
train_path = 'data/input/train.csv'
test_path = 'data/input/test.csv'

df = pd.read_csv(train_path)

df_test = pd.read_csv(test_path)
df = df.sample(frac=0.5, random_state=42)
print(f"Sampled Training Records : {len(df)}")

df.drop(['id'], inplace=True, axis=1)
df['labels'] = df.iloc[:, 1:].values.tolist()
df.drop(df.columns.values[1:-1].tolist(), inplace=True, axis=1)

df["comment_text"] = df["comment_text"].str.lower()
df["comment_text"] = df["comment_text"].str.replace("\xa0", " ", regex=False).str.split().str.join(" ")

df.head()

df_train, df_val = train_test_split(df, test_size=0.1)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

Sampled Training Records : 79786


In [4]:
df.head()

,comment_text,labels
119105,"geez, are you forgetful! we've already discuss...","[0, 0, 0, 0, 0, 0]"
131631,carioca rfa thanks for your support on my requ...,"[0, 0, 0, 0, 0, 0]"
125326,""" birthday no worries, it's what i do ;)enjoy ...","[0, 0, 0, 0, 0, 0]"
111256,pseudoscience category? i'm assuming that this...,"[0, 0, 0, 0, 0, 0]"
83590,"(and if such phrase exists, it would be provid...","[0, 0, 0, 0, 0, 0]"


### Training Parameters

In [5]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory for model predictions and checkpoints
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

MAX_LEN = 200

## Defining the Model

In [6]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

## Preparing the Dataset and Dataloader

### ToxicCommentsDataset Dataset Class
* This class accepts the tokenizer, dataframe, max length, and evaluation mode and generates tokenized output that will be used by the DistilBERT model.
* The DistilBERT tokenizer will be used to tokenize the data from the "comment_text" dataframe column.
* This class is used to create 2 datasets, one for training and one for validation. We will be doing an 80-20 split of the data for training and validation, respectively.

In [7]:
class ToxicCommentsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.comment_text
        self.targets = self.data['labels'].values
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
            'labels': torch.tensor(self.targets[index], dtype=torch.float)
}

### Loading Tokenizer and Generating Training Set

In [8]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)
training_set = ToxicCommentsDataset(df_train, tokenizer, MAX_LEN)
validation_set = ToxicCommentsDataset(df_val, tokenizer, MAX_LEN)

### Defining the Trainer

In [9]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=training_set,         # training dataset
    eval_dataset=validation_set             # evaluation dataset
)

### Training the Model

In [10]:
trainer.train()

/home/yohann/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 71807
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 26928


Step,Training Loss
10,0.693400
20,0.685400
30,0.673200
40,0.649600
50,0.603900
60,0.539700
70,0.491900
80,0.407200
90,0.364500
100,0.277700


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-3000
Configuration saved in ./results/checkpoint-3

TrainOutput(global_step=26928, training_loss=0.039592880258459515, metrics={'train_runtime': 7528.952, 'train_samples_per_second': 28.612, 'train_steps_per_second': 3.577, 'total_flos': 1.11477715236648e+16, 'train_loss': 0.039592880258459515, 'epoch': 3.0})

### Saving the Model

In [11]:
model.save_pretrained("./toxic_comment_model")
tokenizer.save_pretrained("./toxic_comment_model")

Configuration saved in ./toxic_comment_model/config.json
Model weights saved in ./toxic_comment_model/pytorch_model.bin
tokenizer config file saved in ./toxic_comment_model/tokenizer_config.json
Special tokens file saved in ./toxic_comment_model/special_tokens_map.json


('./toxic_comment_model/tokenizer_config.json',
 './toxic_comment_model/special_tokens_map.json',
 './toxic_comment_model/vocab.txt',
 './toxic_comment_model/added_tokens.json',
 './toxic_comment_model/tokenizer.json')